In [1]:
import pandas as pd
import torch

There are 13 features taking integer values (mostly count features) and 26 categorical features. The values of the categorical features have been hashed onto 32 bits for anonymization purposes.

In [2]:
integer_features = ['I' + str(i) for i in range(1, 14)]
categorical_features = ["C" + str(i) for i in range(1, 27)]
data_titles = ['lable'] + integer_features + categorical_features
print(data_titles)

['lable', 'I1', 'I2', 'I3', 'I4', 'I5', 'I6', 'I7', 'I8', 'I9', 'I10', 'I11', 'I12', 'I13', 'C1', 'C2', 'C3', 'C4', 'C5', 'C6', 'C7', 'C8', 'C9', 'C10', 'C11', 'C12', 'C13', 'C14', 'C15', 'C16', 'C17', 'C18', 'C19', 'C20', 'C21', 'C22', 'C23', 'C24', 'C25', 'C26']


In [3]:
# load data original data is tool large. used a 50w items data
# train_data = pd.read_csv("data/train.txt", names=data_titles, sep="\t")
# train_data.head()
original_data = pd.read_csv("data/criteo_sample_50w.csv")
original_data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1.0,25,32.0,8.0,1.0,10.0,22.0,7.0,23.0,...,e5ba7672,6fc84bfb,NaN,NaN,5155d8a3,NaN,423fab69,ded4aac9,NaN,NaN
1,0,NaN,-1,4.0,4.0,3348.0,12.0,82.0,7.0,164.0,...,e5ba7672,e7e991cb,21ddcdc9,a458ea53,1826f894,NaN,423fab69,3fdb382b,ea9a246c,be8b4aee
2,0,NaN,1,13.0,3.0,4356.0,17.0,2.0,3.0,10.0,...,07c540c4,7ef5affa,9908ba56,a458ea53,88d27c62,NaN,32c7478e,3fdb382b,001f3601,49d68486
3,0,NaN,28,1.0,NaN,11064.0,51.0,3.0,6.0,45.0,...,07c540c4,63aa00dd,NaN,NaN,424af181,NaN,3a171ecb,869caea3,NaN,NaN
4,0,3.0,-1,2.0,2.0,931.0,25.0,3.0,15.0,18.0,...,776ce399,0f2f9850,26e97973,a458ea53,d7c93a6d,NaN,32c7478e,a9a2ac1a,9b3e8820,3547c540


In [4]:
# remove null
original_data[integer_features] = original_data[integer_features].fillna('-2022', )
original_data[categorical_features] = original_data[categorical_features].fillna(0, )
targets = ['label']
original_data.head()

,label,I1,I2,I3,I4,I5,I6,I7,I8,I9,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,1,1.0,25,32.0,8.0,1.0,10.0,22.0,7.0,23.0,...,e5ba7672,6fc84bfb,0,0,5155d8a3,0,423fab69,ded4aac9,0,0
1,0,-2022,-1,4.0,4.0,3348.0,12.0,82.0,7.0,164.0,...,e5ba7672,e7e991cb,21ddcdc9,a458ea53,1826f894,0,423fab69,3fdb382b,ea9a246c,be8b4aee
2,0,-2022,1,13.0,3.0,4356.0,17.0,2.0,3.0,10.0,...,07c540c4,7ef5affa,9908ba56,a458ea53,88d27c62,0,32c7478e,3fdb382b,001f3601,49d68486
3,0,-2022,28,1.0,-2022,11064.0,51.0,3.0,6.0,45.0,...,07c540c4,63aa00dd,0,0,424af181,0,3a171ecb,869caea3,0,0
4,0,3.0,-1,2.0,2.0,931.0,25.0,3.0,15.0,18.0,...,776ce399,0f2f9850,26e97973,a458ea53,d7c93a6d,0,32c7478e,a9a2ac1a,9b3e8820,3547c540


In [5]:
from sklearn.preprocessing import LabelEncoder

original_data[categorical_features] = original_data[categorical_features].astype('str')
for feature in categorical_features:
    lbe = LabelEncoder()
    # trans each feature to an id
    original_data[feature] = lbe.fit_transform(original_data[feature])
original_data[categorical_features].head()

,C1,C2,C3,C4,C5,C6,C7,C8,C9,C10,...,C17,C18,C19,C20,C21,C22,C23,C24,C25,C26
0,20,10,103950,48164,37,6,3796,18,2,21865,...,9,1541,0,0,43790,0,4,25283,0,0
1,20,249,108293,58867,37,13,4961,165,2,15314,...,9,3188,212,2,13014,0,4,7205,54,16624
2,383,168,109278,43314,37,13,3938,18,2,9543,...,0,1727,1037,2,73724,0,2,7205,1,6343
3,760,312,84271,68069,160,6,9868,18,2,12169,...,0,1371,0,0,35768,0,3,15271,0,0
4,760,451,17999,16618,37,6,4309,410,2,5368,...,5,217,242,2,116420,0,2,19260,36,4586


In [6]:
# normalization
original_data[integer_features] = original_data[integer_features].astype('float')
for feature in integer_features:
    mean = original_data[feature].mean()
    std = original_data[feature].std()
    original_data[feature] = (original_data[feature] - mean) / (std + 1e-12)  #remove 0

original_data[integer_features].head()


,I1,I2,I3,I4,I5,I6,I7,I8,I9,I10,I11,I12,I13
0,0.852800,-0.192994,0.517242,0.589790,-0.271033,0.368287,0.222795,-0.073269,0.004389,0.855558,0.221647,-0.549225,0.588038
1,-1.169387,-0.269791,0.487845,0.585280,-0.220090,0.370272,0.368872,-0.073269,0.298103,-1.169416,0.236494,1.819670,0.583535
2,-1.169387,-0.263883,0.497294,0.584153,-0.204748,0.375234,0.174102,-0.132192,-0.022691,-1.169416,0.204325,-0.549225,0.582409
3,-1.169387,-0.184132,0.484695,-1.698848,-0.102649,0.408977,0.176537,-0.088000,0.050217,-1.169416,0.204325,-0.549225,-1.697306
4,0.854800,-0.269791,0.485745,0.583026,-0.256878,0.383173,0.176537,0.044577,-0.006026,0.855558,0.204325,-0.549225,0.581284


In [7]:
original_data.shape

(500000, 40)

In [8]:
from sklearn.model_selection import train_test_split

train, test = train_test_split(original_data, test_size=0.2, random_state=2248)
print(train.shape, test.shape)

(400000, 40) (100000, 40)


In [9]:
import torch.utils.data as Data

train_data = Data.TensorDataset(torch.LongTensor(train[categorical_features].values),
                                torch.FloatTensor(train[integer_features].values),
                                torch.FloatTensor(train['label'].values), )

train_loader = Data.DataLoader(dataset=train_data, batch_size=2048, shuffle=True)


In [10]:
test_data = Data.TensorDataset(torch.LongTensor(test[categorical_features].values),
                               torch.FloatTensor(test[integer_features].values),
                               torch.FloatTensor(test['label'].values), )
test_loader = Data.DataLoader(dataset=test_data, batch_size=2048, shuffle=False)

In [11]:
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

In [12]:
device

device(type='cuda')

In [13]:
cate_each_features_size = [original_data[f].nunique() for f in categorical_features]
cate_each_features_size

[1036,
 526,
 177397,
 75762,
 234,
 14,
 10165,
 482,
 3,
 23499,
 4501,
 152106,
 3029,
 26,
 7825,
 120291,
 10,
 3512,
 1704,
 4,
 138137,
 14,
 15,
 29183,
 63,
 22354]

In [14]:
# init model
from DeepFM import DeepFM

In [15]:
model = DeepFM(39, integer_features, categorical_features, cate_each_features_size,device='cuda')
model.to(device)

DeepFM(
  (fm_1st_int_vec): Linear(in_features=13, out_features=1, bias=True)
  (fm_1st_cate_emb): ModuleList(
    (0): Embedding(1036, 1)
    (1): Embedding(526, 1)
    (2): Embedding(177397, 1)
    (3): Embedding(75762, 1)
    (4): Embedding(234, 1)
    (5): Embedding(14, 1)
    (6): Embedding(10165, 1)
    (7): Embedding(482, 1)
    (8): Embedding(3, 1)
    (9): Embedding(23499, 1)
    (10): Embedding(4501, 1)
    (11): Embedding(152106, 1)
    (12): Embedding(3029, 1)
    (13): Embedding(26, 1)
    (14): Embedding(7825, 1)
    (15): Embedding(120291, 1)
    (16): Embedding(10, 1)
    (17): Embedding(3512, 1)
    (18): Embedding(1704, 1)
    (19): Embedding(4, 1)
    (20): Embedding(138137, 1)
    (21): Embedding(14, 1)
    (22): Embedding(15, 1)
    (23): Embedding(29183, 1)
    (24): Embedding(63, 1)
    (25): Embedding(22354, 1)
  )
  (fm_2nd_cate_emb): ModuleList(
    (0): Embedding(1036, 8)
    (1): Embedding(526, 8)
    (2): Embedding(177397, 8)
    (3): Embedding(75762, 8)
  

In [16]:
print("parameters", sum(pa.numel() for pa in model.parameters()))

parameters 7037251


In [17]:
# train and eval
model.fit(train_loader, test_loader, epochs=50)

1669556079.104278
Epoch:1, Step:50/196, Loss:20.14670824050903
Epoch:1, Step:100/196, Loss:17.401493225097656
Epoch:1, Step:150/196, Loss:14.837036097844441
Epoch:1, Step:196/196, Loss:12.721613014230924
Best AUC:0.6515834713560766
1669556085.5180995
Epoch:2, Step:50/196, Loss:2.5678839707374572
Epoch:2, Step:100/196, Loss:1.9238552033901215
Epoch:2, Step:150/196, Loss:1.5866546853383383
Epoch:2, Step:196/196, Loss:1.4006596162002913
Best AUC:0.6647373803188643
1669556090.9835365
Epoch:3, Step:50/196, Loss:0.6435158598423004
Epoch:3, Step:100/196, Loss:0.6213021522760391
Epoch:3, Step:150/196, Loss:0.6025592776139578
Epoch:3, Step:196/196, Loss:0.5908416252659292
Best AUC:0.7073969730364735
1669556096.4097717
Epoch:4, Step:50/196, Loss:0.5329968571662903
Epoch:4, Step:100/196, Loss:0.5319780048727989
Epoch:4, Step:150/196, Loss:0.5271916230519612
Epoch:4, Step:196/196, Loss:0.5245649597170402
Best AUC:0.7324031262428
1669556101.6785994
Epoch:5, Step:50/196, Loss:0.5011488801240921
Epoc

In [15]:
from FM import FactorizationMachine
base_model = FactorizationMachine(39, integer_features, categorical_features, cate_each_features_size,device='cuda')
base_model.to(device)

FactorizationMachine(
  (fm_1st_int_vec): Linear(in_features=13, out_features=1, bias=True)
  (fm_1st_cate_emb): ModuleList(
    (0): Embedding(1036, 1)
    (1): Embedding(526, 1)
    (2): Embedding(177397, 1)
    (3): Embedding(75762, 1)
    (4): Embedding(234, 1)
    (5): Embedding(14, 1)
    (6): Embedding(10165, 1)
    (7): Embedding(482, 1)
    (8): Embedding(3, 1)
    (9): Embedding(23499, 1)
    (10): Embedding(4501, 1)
    (11): Embedding(152106, 1)
    (12): Embedding(3029, 1)
    (13): Embedding(26, 1)
    (14): Embedding(7825, 1)
    (15): Embedding(120291, 1)
    (16): Embedding(10, 1)
    (17): Embedding(3512, 1)
    (18): Embedding(1704, 1)
    (19): Embedding(4, 1)
    (20): Embedding(138137, 1)
    (21): Embedding(14, 1)
    (22): Embedding(15, 1)
    (23): Embedding(29183, 1)
    (24): Embedding(63, 1)
    (25): Embedding(22354, 1)
  )
  (fm_2nd_cate_emb): ModuleList(
    (0): Embedding(1036, 8)
    (1): Embedding(526, 8)
    (2): Embedding(177397, 8)
    (3): Embeddin

In [16]:
base_model.fit(train_loader, test_loader, epochs=50)

1669557175.3757775
Epoch:1, Step:50/196, Loss:20.744446754455566
Epoch:1, Step:100/196, Loss:17.9903546333313
Epoch:1, Step:150/196, Loss:16.136262035369874
Epoch:1, Step:196/196, Loss:14.585182112090441
Best AUC:0.6270172752419328
1669557182.1511912
Epoch:2, Step:50/196, Loss:5.952697582244873
Epoch:2, Step:100/196, Loss:4.551464211940766
Epoch:2, Step:150/196, Loss:3.605742723941803
Epoch:2, Step:196/196, Loss:3.0231840954143174
Best AUC:0.6353366201333568
1669557187.1526775
Epoch:3, Step:50/196, Loss:0.8065355741977691
Epoch:3, Step:100/196, Loss:0.7456582975387573
Epoch:3, Step:150/196, Loss:0.7003594823678334
Epoch:3, Step:196/196, Loss:0.6703778359354758
Best AUC:0.6796058202932731
1669557192.2644851
Epoch:4, Step:50/196, Loss:0.5311242336034775
Epoch:4, Step:100/196, Loss:0.526192232966423
Epoch:4, Step:150/196, Loss:0.5233119157950084
Epoch:4, Step:196/196, Loss:0.5204556411018177
Best AUC:0.7201034137818729
1669557197.209048
Epoch:5, Step:50/196, Loss:0.49944277465343473
Epoch